In [9]:
import pandas as pd
from sqlalchemy import create_engine, text
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# CSV файл унших
df = pd.read_csv("C:/Users/Dell/Downloads/Ecn325 data.csv")

# 1. Хүснэгтүүдийг хуваах
locations_df = df[['id', 'state_code', 'latitude', 'longitude', 'zip_code', 'city', 
                   'is_CA', 'is_NY', 'is_MA', 'is_TX', 'is_otherstate']].copy()

funding_df = df[['id', 'funding_rounds', 'funding_total_usd', 'milestones',
                 'has_VC', 'has_angel', 'has_roundA', 'has_roundB', 'has_roundC',
                 'has_roundD', 'avg_participants']].copy()

startup_profile_df = df[['id', 'category_code', 'is_software', 'is_web', 'is_mobile',
                         'is_enterprise', 'is_advertising', 'is_gamesvideo', 'is_ecommerce',
                         'is_biotech', 'is_consulting', 'is_othercategory', 'is_top500', 
                         'status']].copy()

# 2. SQLite өгөгдлийн сан үүсгэх
engine = create_engine('sqlite:///startup_ml.db', echo=False)

locations_df.to_sql('locations', con=engine, if_exists='replace', index=False)
funding_df.to_sql('funding', con=engine, if_exists='replace', index=False)
startup_profile_df.to_sql('startup_profile', con=engine, if_exists='replace', index=False)

print("✅ 3 хүснэгттэй өгөгдлийн сан үүссэн.")

# 3. ML-д зориулж бүх өгөгдлийг нэгтгэх
ml_df = df.copy()

# Target-г бэлтгэх — "status" баганаас амжилтыг гаргаж авах
ml_df['Success'] = ml_df['status'].apply(lambda x: 1 if str(x).lower() in ['acquired', 'operating'] else 0)

#  Features сонгох (жишээ)
features = ['funding_rounds', 'funding_total_usd', 'milestones',
            'has_VC', 'has_angel', 'has_roundA', 'has_roundB', 'has_roundC', 'has_roundD',
            'is_top500', 'avg_participants']

X = ml_df[features]
y = ml_df['Success']

#  Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#  ML Model — Logistic Regression
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

#  Үр дүн
print("📈 Logistic Regression-ийн үр дүн:")
print(classification_report(y_test, y_pred))


✅ 3 хүснэгттэй өгөгдлийн сан үүссэн.
📈 Logistic Regression-ийн үр дүн:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        73
           1       0.61      1.00      0.75       112

    accuracy                           0.61       185
   macro avg       0.30      0.50      0.38       185
weighted avg       0.37      0.61      0.46       185



c:\Users\Dell\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Dell\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Dell\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier

In [8]:
import os

db_path = os.path.abspath("startup_ml.db")
print("📁 Database-ийн байрлал:", db_path)


📁 Database-ийн байрлал: c:\Users\Dell\AppData\Local\Programs\Microsoft VS Code\startup_ml.db
